In [2]:
import os
import numpy as np

import itk
from itk import TubeTK as ttk

from itkwidgets import view

In [12]:
DWI_path = '/media/peirong/PR/IXI/Processed/IXI002-Guys-0828/DiffusionMaps/IXI002-Guys-0828-DTI-'
DWI0_path = DWI_path + '00.nii.gz'
DWI1_path = DWI_path + '01.nii.gz'
DWI2_path = DWI_path + '02.nii.gz'
DWI3_path = DWI_path + '03.nii.gz'
DWI4_path = DWI_path + '04.nii.gz'
DWI5_path = DWI_path + '05.nii.gz'

MRA_path = '/media/peirong/PR/IXI/Processed/IXI002-Guys-0828/AdvectionMaps/IXI002-Guys-0828-MRA.nii.gz'
T1_path = '/media/peirong/PR/IXI/IXI-T1/IXI002-Guys-0828-T1.nii.gz'
T2_path = '/media/peirong/PR/IXI/IXI-T2/IXI002-Guys-0828-T2.nii.gz'

save_fld = '/home/peirong/Desktop'

In [13]:
ImageType = itk.Image[itk.F, 3]
ReaderType = itk.ImageFileReader[ImageType]

reader1 = ReaderType.New(FileName=DWI0_path)
reader1.Update()
im_dwi0 = reader1.GetOutput()
reader1 = ReaderType.New(FileName=DWI1_path)
reader1.Update()
im_dwi1 = reader1.GetOutput()
reader1 = ReaderType.New(FileName=DWI2_path)
reader1.Update()
im_dwi2 = reader1.GetOutput()
reader1 = ReaderType.New(FileName=DWI3_path)
reader1.Update()
im_dwi3 = reader1.GetOutput()
reader1 = ReaderType.New(FileName=DWI4_path)
reader1.Update()
im_dwi4 = reader1.GetOutput()
reader1 = ReaderType.New(FileName=DWI5_path)
reader1.Update()
im_dwi5 = reader1.GetOutput()

reader2 = ReaderType.New(FileName=MRA_path)
reader2.Update()
im_mra = reader2.GetOutput()

reader3 = ReaderType.New(FileName=T1_path)
reader3.Update()
im_t1 = reader3.GetOutput()

reader4 = ReaderType.New(FileName=T2_path)
reader4.Update()
im_t2 = reader4.GetOutput()

In [17]:
# Resample MRA into isotropic voxels and register DWI, T1, T2 with isotropic MRA

res = ttk.ResampleImage.New(Input = im_mra) 
#res.SetMakeHighResIso(True)
res.SetSpacing([1., 1., 1])
res.Update()
im_mra_iso = res.GetOutput()
itk.imwrite(im_mra_iso, os.path.join(save_fld, "MRA-Iso.mha"))

def register_images(moving_img, fixed_img, save_path = None):
    imReg = ttk.RegisterImages[ImageType].New()  # standard protocol for within patient within visit registration
    imReg.SetFixedImage(fixed_img)
    imReg.SetMovingImage(moving_img)
    imReg.SetReportProgress(True)
    imReg.SetExpectedOffsetMagnitude(40)
    imReg.SetExpectedRotationMagnitude(0.01)
    imReg.SetExpectedScaleMagnitude(0.01)
    imReg.SetRigidMaxIterations(500)
    imReg.SetRigidSamplingRatio(0.1)
    imReg.SetRegistration("RIGID")
    imReg.SetMetric("MATTES_MI_METRIC")
    imReg.Update() 
    # NOTE: interpolation arg to be chosen
    registered_moving_img = imReg.GetFinalMovingImage("LINEAR_INTERPOLATION")
    if save_path is not None:
        itk.imwrite(registered_moving_img, save_path)
        return registered_moving_img, save_path
    return registered_moving_img

im_dwi0_2mra_iso, _ = register_images(moving_img = im_dwi0, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI0_2MRA-Iso.mha"))
im_dwi1_2mra_iso, _ = register_images(moving_img = im_dwi1, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI1_2MRA-Iso.mha"))
im_dwi2_2mra_iso, _ = register_images(moving_img = im_dwi2, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI2_2MRA-Iso.mha"))
im_dwi3_2mra_iso, _ = register_images(moving_img = im_dwi3, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI3_2MRA-Iso.mha"))
im_dwi4_2mra_iso, _ = register_images(moving_img = im_dwi4, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI4_2MRA-Iso.mha"))
im_dwi5_2mra_iso, _ = register_images(moving_img = im_dwi5, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "DWI5_2MRA-Iso.mha"))

im_t1_2mra_iso, _ = register_images(moving_img = im_t1, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "T1_2MRA-Iso.mha"))
im_t2_2mra_iso, _ = register_images(moving_img = im_t2, fixed_img = im_mra_iso, \
                                  save_path = os.path.join(save_fld, "T2_2MRA-Iso.mha"))

In [12]:
# Plot for checking #
CheckerboardImagesType = itk.CheckerBoardImageFilter[ImageType]
cbt1t2= CheckerboardImagesType.New(Input1=im_t1_2mra_iso, Input2=im_t2_2mra_iso)
cbt1t2.Update()
imt1t2 = ImageType.New()
imt1t2 = cbt1t2.GetOutput()
view(imt1t2)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [19]:
N = 8
BaseLineFld = '/media/peirong/PR/TubeTK_Data'
readerList = ["003", "010", "026", "034", "045", "056", "063", "071"]

imBase = []
imBaseB = []
for i in range(0,N):
    name = os.path.join(BaseLineFld, "Normal"+readerList[i]+"-FLASH.mha")
    nameB = os.path.join(BaseLineFld, "Normal"+readerList[i]+"-FLASH-Brain.mha")
    reader = ReaderType.New(FileName=name)
    reader.Update()
    imBaseTmp = reader.GetOutput()
    reader = ReaderType.New(FileName=nameB)
    reader.Update()
    imBaseBTmp = reader.GetOutput()
    imBase.append(imBaseTmp)
    imBaseB.append(imBaseBTmp)

In [20]:
imMath = ttk.ImageMath.New(Input = im_t1_2mra_iso)
imMath.Blur(2)
im_t1_2mra_isoBlur = imMath.GetOutput()

regB = []
regBB = []
#print('Start')
for i in range(0,N):
    print(i)
    imMath.SetInput(imBase[i])
    imMath.Blur(2)
    imBaseBlur = imMath.GetOutput()
    regBTo1 = ttk.RegisterImages[ImageType].New(FixedImage = im_t1_2mra_isoBlur, MovingImage = imBaseBlur)
    regBTo1.SetReportProgress(True)
    regBTo1.SetExpectedOffsetMagnitude(40)
    regBTo1.SetExpectedRotationMagnitude(0.01)
    regBTo1.SetExpectedScaleMagnitude(0.1)
    regBTo1.SetRigidMaxIterations(500)
    regBTo1.SetAffineMaxIterations(500)
    regBTo1.SetRigidSamplingRatio(0.1)
    regBTo1.SetAffineSamplingRatio(0.1)
    regBTo1.SetInitialMethodEnum("INIT_WITH_IMAGE_CENTERS")
    regBTo1.SetRegistration("PIPELINE_AFFINE")
    regBTo1.SetMetric("MATTES_MI_METRIC")
    #regBTo1.SetMetric("NORMALIZED_CORRELATION_METRIC") - Really slow!
    #regBTo1.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    regBTo1.Update()
    img = regBTo1.ResampleImage("LINEAR", imBase[i])
    regB.append( img )
    img = regBTo1.ResampleImage("LINEAR", imBaseB[i])
    regBB.append( img )

Start
0
1
2
3
4
5
6
7


In [22]:
regBBT = []
for i in range(0,N):
    imMath = ttk.ImageMath[ImageType,ImageType].New( Input = regBB[i] )
    imMath.Threshold(0,1,0,1)
    img = imMath.GetOutput()
    if i==0:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( img )
        imMathSum.AddImages( img, 1.0/N, 0 )
        sumBBT = imMathSum.GetOutput()
    else:
        imMathSum = ttk.ImageMath[ImageType,ImageType].New( sumBBT )
        imMathSum.AddImages( img, 1, 1.0/N )
        sumBBT = imMathSum.GetOutput()
        
#view(sumBBT)

In [25]:
insideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
insideMath.Threshold(1,1,1,0)
insideMath.Erode(5,1,0)
brainInside = insideMath.GetOutput()

outsideMath = ttk.ImageMath[ImageType,ImageType].New( Input = sumBBT )
outsideMath.Threshold(0,0,1,0)
outsideMath.Erode(10,1,0)
brainOutsideAll = outsideMath.GetOutput()
outsideMath.Erode(20,1,0)
outsideMath.AddImages(brainOutsideAll, -1, 1)
brainOutside = outsideMath.GetOutput()

outsideMath.AddImages(brainInside,1,2)
brainCombinedMask = outsideMath.GetOutputUChar()

outsideMath.AddImages(im_t1_2mra_iso, 512, 1)

brainCombinedMaskView = outsideMath.GetOutput()
#view(brainCombinedMaskView) # Plotting #

In [27]:
LabelMapType = itk.Image[itk.UC,3]

segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[ImageType,LabelMapType].New()
segmenter.SetFeatureImage( im_mra_iso )
segmenter.AddFeatureImage( im_t1_2mra_iso )
segmenter.AddFeatureImage( im_t2_2mra_iso )
segmenter.SetInputLabelMap( brainCombinedMask )
segmenter.SetObjectId( 2 )
segmenter.AddObjectId( 1 )
segmenter.SetVoidId( 0 )
segmenter.SetErodeDilateRadius( 5 )
segmenter.Update()
segmenter.ClassifyImages()
brainCombinedMaskClassified = segmenter.GetOutputLabelMap()

In [36]:
cast = itk.CastImageFilter[LabelMapType, ImageType].New()
cast.SetInput(brainCombinedMaskClassified)
cast.Update()
brainMaskF = cast.GetOutput()

brainMath = ttk.ImageMath[ImageType,ImageType].New(Input = brainMaskF)
brainMath.Threshold(2,2,1,0)
brainMath.Dilate(2,1,0)
brainMaskD = brainMath.GetOutput()

brainMath.SetInput( im_mra_iso )
brainMath.ReplaceValuesOutsideMaskRange( brainMaskD, 1, 1, 0)
brain = brainMath.GetOutput()

#view(brain)

In [41]:
itk.imwrite(brainMaskD, os.path.join(save_fld, "MRAMask-Brain.mha"))


itk.imwrite(brain, os.path.join(save_fld, "MRA-Brain.mha"))

def masking(masker, img, mask, save_path = None):
    masker.SetInput(img)
    masker.ReplaceValuesOutsideMaskRange( mask, 1, 1, 0 )
    masked_img = masker.GetOutput()
    if save_path is not None:
        itk.imwrite(masked_img, save_path)
        return masked_img, save_path
    return masked_img

masked_im_t1_2mra_iso, _ = masking(brainMath, im_t1_2mra_iso, brainMaskD, os.path.join(save_fld, "T1_2MRA-Brain.mha"))
masked_im_t2_2mra_iso, _ = masking(brainMath, im_t2_2mra_iso, brainMaskD, os.path.join(save_fld, "T2_2MRA-Brain.mha"))

masked_im_dwi0_2mra_iso, _ = masking(brainMath, im_dwi0_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI0_2MRA-Brain.mha"))
masked_im_dwi1_2mra_iso, _ = masking(brainMath, im_dwi1_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI1_2MRA-Brain.mha"))
masked_im_dwi2_2mra_iso, _ = masking(brainMath, im_dwi2_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI2_2MRA-Brain.mha"))
masked_im_dwi3_2mra_iso, _ = masking(brainMath, im_dwi3_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI3_2MRA-Brain.mha"))
masked_im_dwi4_2mra_iso, _ = masking(brainMath, im_dwi4_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI4_2MRA-Brain.mha"))
masked_im_dwi5_2mra_iso, _ = masking(brainMath, im_dwi5_2mra_iso, brainMaskD, os.path.join(save_fld, "DWI5_2MRA-Brain.mha"))